In [ ]:
import numpy as np
import torch
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

def model(t_u, w, b):
    return w * t_u + b


def loss_fn(t_p, t_u):
    squared_diffs = (t_p - t_u) ** 2
    return squared_diffs.mean()

#requires_grad = True params를 조상으로 두는 모든 텐서는 params에 접근할 수 있도록 허용
params = torch.tensor([1.0, 0.0] , requires_grad = True) 
learning_rate = 1e-5

def training_loop(n_epochs , learning_rate , params, t_u, t_c):
    for epoch in range(1,n_epochs+1):
        if params.grad is not None:
            params.grad.zero_()

        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()

        with torch.no_grad():
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print(f"Epoch : {epoch}  , Loss : {loss} , Grad  : {params.grad}")

    return params



t_un = t_u * 0.1




### 경사 하강 옵티마이저 사용
- 여기서 사용하는 SGD는 확룰적 경사 하강의 약자이다
- SGD
    - 미니배치라고 불리는 여러 샘플 중에서 임의로 뽑은 일부에 대해 평균을 계산에 얻음

In [ ]:
import torch.optim as optim

dir(optim)

In [ ]:
optimizer = optim.SGD([params], lr = learning_rate)

### 여기서 즁요한 것은 기울기값을 0 으로 초기화해준다
- optimizer.zero_grad() 

In [ ]:
params = torch.tensor([1.0, 0.0] , requires_grad = True) 
learning_rate = 1e-2
optimizer = optim.SGD([params], lr = learning_rate)

t_p = model(t_un, *params)
loss = loss_fn(t_p,t_c)

optimizer.zero_grad()
loss.backward()
optimizer.step()

params

### 옵티마이저를 사용한 training_loop 만들기

In [ ]:
def training_loop(n_epochs , optimizer , params, t_u, t_c):
    for epoch in range(1,n_epochs+1):

        t_p = model(t_u, *params)
        loss = loss_fn(t_p,t_c)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch : {epoch}  , Loss : {loss} , Grad  : {params.grad}")

    return params




In [ ]:
params = torch.tensor([1.0, 0.0] , requires_grad = True) 
learning_rate = 1e-2
optimizer = optim.SGD([params], lr = learning_rate)

In [ ]:
training_loop(
    n_epochs  = 5000, 
    optimizer = optimizer, 
    params = params, 
    t_u = t_un, 
    t_c = t_c)

### SDG 말고 Adam 으로 학습해보기
- 이렇게 옵티마이저를 변경하는 것 뿐만이 아니라 모델을 변경하는 식으로 유연하게 사용하 수 있다

In [ ]:
params = torch.tensor([1.0, 0.0] , requires_grad = True) 
learning_rate = 1e-2
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(
    n_epochs  = 2000, 
    optimizer = optimizer, 
    params = params, 
    t_u = t_u, 
    t_c = t_c)